In [ ]:
%pip install -qU langchain-google-vertexai
%pip install -qU langchain-community
%pip install -qU faiss-cpu
%pip install -qU tiktoken
#

  # google-cloud-aiplatform \
  # google-cloud-storage \



In [ ]:
!gcloud config set project fiscalia-455807


Updated property [core/project].


In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings

# Initialize the a specific Embeddings Model version
embeddings = VertexAIEmbeddings(model_name="text-embedding-004")

In [ ]:
from google.cloud import storage
from langchain_community.vectorstores import FAISS
import os

def download_faiss_from_gcs(bucket_name: str, gcs_prefix: str, local_path: str):
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    # Crée le dossier local
    os.makedirs(local_path, exist_ok=True)

    # Liste et télécharge les fichiers du vector store
    blobs = list(bucket.list_blobs(prefix=gcs_prefix))
    for blob in blobs:
        filename = blob.name.split("/")[-1]
        destination = os.path.join(local_path, filename)
        blob.download_to_filename(destination)
        print(f"⬇️  {blob.name} → {destination}")

# === Paramètres
BUCKET_NAME = "fiscalia-455807-vector-store"

# CGI
CGI_PREFIX = "vector_stores/cgi_index"
CGI_LOCAL_PATH = "/tmp/vector_stores/cgi_faiss"

# BOFIP
BOFIP_PREFIX = "vector_stores/bofip_index"
BOFIP_LOCAL_PATH = "/tmp/vector_stores/bofip_faiss"

# BOFIP_bareme
BOFIP_BAREME_PREFIX = "vector_stores/bofip_bareme_index"
BOFIP_BAREME_LOCAL_PATH = "/tmp/vector_stores/bofip_bareme_faiss"

# === Téléchargement & Chargement CGI
download_faiss_from_gcs(BUCKET_NAME, CGI_PREFIX, CGI_LOCAL_PATH)
cgi_db = FAISS.load_local(
    CGI_LOCAL_PATH,
    embeddings,
    allow_dangerous_deserialization=True
)

# === Téléchargement & Chargement BOFIP
download_faiss_from_gcs(BUCKET_NAME, BOFIP_PREFIX, BOFIP_LOCAL_PATH)
bofip_db = FAISS.load_local(
    CGI_LOCAL_PATH,
    embeddings,
    allow_dangerous_deserialization=True
)

print("✅ Vector store chargé depuis GCS")

# === Téléchargement & Chargement BOFIP_bareme
download_faiss_from_gcs(BUCKET_NAME, BOFIP_BAREME_PREFIX, BOFIP_BAREME_LOCAL_PATH)
bofip_barem_db = FAISS.load_local(
    CGI_LOCAL_PATH,
    embeddings,
    allow_dangerous_deserialization=True
)

print("✅ Vector store chargé depuis GCS")

⬇️  vector_stores/cgi_index/index.faiss → /tmp/vector_stores/cgi_faiss/index.faiss
⬇️  vector_stores/cgi_index/index.pkl → /tmp/vector_stores/cgi_faiss/index.pkl
⬇️  vector_stores/bofip_index/index.faiss → /tmp/vector_stores/bofip_faiss/index.faiss
⬇️  vector_stores/bofip_index/index.pkl → /tmp/vector_stores/bofip_faiss/index.pkl
✅ Vector store chargé depuis GCS
⬇️  vector_stores/bofip_bareme_index/index.faiss → /tmp/vector_stores/bofip_bareme_faiss/index.faiss
⬇️  vector_stores/bofip_bareme_index/index.pkl → /tmp/vector_stores/bofip_bareme_faiss/index.pkl
✅ Vector store chargé depuis GCS


In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain_google_vertexai import VertexAI
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

# === LLM
llm = VertexAI(model_name="gemini-2.0-flash-lite", temperature=0.3)

# === Prompt générique
prompt_template = """
Tu es un assistant fiscal expert du Code Général des Impôts et du BOFiP.

Tu dois répondre à la question suivante de manière claire, structurée et précise.
Ta réponse doit **prioritairement** s’appuyer sur les extraits de documents fournis.
Tu peux utiliser tes **connaissances fiscales générales** si les extraits sont insuffisants, à condition de l’indiquer explicitement dans ta réponse.

Lorsque les extraits contiennent des tableaux, tu es autorisé à les interpréter et à appliquer les formules qui y figurent.

Exemple :
---
Extrait :
Puissance administrative | Jusqu'à 5 000 km | De 5001 à 20 000 km | Au delà de 20 000 km
7 CV et plus | d x 0,697 | (d x 0,394) + 1515 | d x 0,470

Question :
Quel est le montant des indemnités kilométriques pour 7 CV et 5500 km ?

Réponse :
Pour un véhicule de 7 CV ayant parcouru 5500 km, la formule applicable est : (5500 × 0,394) + 1515 = 3 682 €.

---

Contexte :
{context}

Question :
{question}

Réponse :
"""


prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


In [ ]:
from langchain.chains import RetrievalQA

def build_rag_chain(retriever):
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt}
    )


In [ ]:
# === 1. CGI
retriever_cgi = cgi_db.as_retriever(search_kwargs={"k": 3})
qa_chain_cgi = build_rag_chain(retriever_cgi)

def rag_cgi(query: str) -> str:
    result = qa_chain_cgi.invoke(query)
    sources = [doc.metadata.get("source", "CGI") for doc in result["source_documents"]]
    return f"{result['result']}\n\n📚 Sources CGI :\n" + "\n".join(f"- {s}" for s in sources)

# === 2. BOFiP
retriever_bofip = bofip_db.as_retriever(search_kwargs={"k": 3})
qa_chain_bofip = build_rag_chain(retriever_bofip)

def rag_bofip(query: str) -> str:
    result = qa_chain_bofip.invoke(query)
    sources = [doc.metadata.get("source", "BOFiP") for doc in result["source_documents"]]
    return f"{result['result']}\n\n📚 Sources BOFiP :\n" + "\n".join(f"- {s}" for s in sources)

# === 3. BOFiP_bareme
retriever_bofip_barem = bofip_barem_db.as_retriever(search_kwargs={"k": 3})
qa_chain_bofip_barem = build_rag_chain(retriever_bofip_barem)

def rag_bofip_barem(query: str) -> str:
    result = qa_chain_bofip_barem.invoke(query)
    sources = [doc.metadata.get("source", "BOFiP Barème") for doc in result["source_documents"]]
    return f"{result['result']}\n\n📚 Sources BOFiP Barème :\n" + "\n".join(f"- {s}" for s in sources)



In [ ]:
def search_web(query: str) -> str:
    return "🔍 Recherche web désactivée pour l’instant."


In [ ]:
from langchain.agents import Tool

tools = [
    Tool(
        name="Recherche_CGI",
        func=rag_cgi,
        description="Utilise pour répondre à une question en interrogeant le Code Général des Impôts."
    ),
    Tool(
        name="Recherche_BOFIP",
        func=rag_bofip,
        description="Utilise pour répondre à une question en interrogeant la base de doctrine fiscale (BOFiP)."
    ),
      Tool(
        name="Recherche_BOFIP_bareme",
        func=rag_bofip,
        description="Utilise pour répondre à une question en interrogeant les baremes de la base de doctrine fiscale (BOFiP)."
    ),
    Tool(
        name="Recherche_Web",
        func=search_web,
        description="(Optionnel) Recherche complémentaire sur des sites spécialisés de fiscalité."
    )
]


In [ ]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# === Test
response = agent.invoke("Comment fonctionne les indemnités kilométriques ?")
print("🧠 Réponse de l’agent :", response)




> Entering new AgentExecutor chain...
Je dois trouver des informations sur le fonctionnement des indemnités kilométriques. Je vais commencer par chercher dans le Code Général des Impôts.
Action: Recherche_CGI
Action Input: indemnités kilométriques fonctionnement
Observation: Je suis désolé, mais les extraits fournis ne contiennent aucune information concernant le calcul ou le fonctionnement des indemnités kilométriques. Ils traitent de sujets liés aux impôts directs, aux provisions techniques dans le secteur de l'assurance, et à la compensation des résultats techniques déficitaires.

Par conséquent, je ne peux pas répondre à votre question sur le fonctionnement des indemnités kilométriques en me basant sur les documents fournis.


📚 Sources CGI :
- Code Général des Impôts
- Code Général des Impôts
- Code Général des Impôts
Thought:Je n'ai pas trouvé d'informations pertinentes dans le Code Général des Impôts. Je vais maintenant chercher dans la base de doctrine fiscale (BOFiP).
Action

In [ ]:
# === Test
response = agent.invoke("Combien je vais payer d'indemnités kilométriques pour une voiture de 7 CV et 5500 km?")
print("🧠 Réponse de l’agent :", response)



> Entering new AgentExecutor chain...
Je dois calculer le montant des indemnités kilométriques. Pour cela, je dois consulter les barèmes fiscaux. Je vais commencer par chercher le barème applicable pour les voitures.
Action: Recherche_BOFIP_bareme
Action Input: indemnités kilométriques barème voiture
Observation: Les informations fournies ne permettent pas de calculer directement les indemnités kilométriques en utilisant le barème. Cependant, elles donnent des indications importantes :

1.  **Le barème existe :** Il est précisé que l'évaluation des frais de déplacement (autres que les frais de péage, de garage, de parking et les intérêts d'emprunt) peut se faire sur la base d'un barème forfaitaire fixé par arrêté du ministre chargé du budget.
2.  **Les critères du barème :** Ce barème tient compte de la puissance administrative du véhicule (limitée à 7 CV), du type de motorisation et de la distance annuelle parcourue.
3.  **Option pour les frais réels :** Les bénéficiaires de traitem

NameError: name 'chunks_bofip_bareme' is not defined